In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
cov = pd.read_csv('covid_us_county.csv')

In [14]:
pop = pd.read_csv('us_county.csv')
pop

,fips,county,state,state_code,male,female,median_age,population,female_percentage,lat,long
0,1001,Autauga County,Alabama,AL,26874,28326,37.8,55200,51.315217,32.534923,-86.642730
1,1003,Baldwin County,Alabama,AL,101188,106919,42.8,208107,51.376936,30.727479,-87.722564
2,1005,Barbour County,Alabama,AL,13697,12085,39.9,25782,46.873788,31.869581,-85.393210
3,1007,Bibb County,Alabama,AL,12152,10375,39.9,22527,46.055844,32.998628,-87.126475
4,1009,Blount County,Alabama,AL,28434,29211,40.8,57645,50.673953,33.980869,-86.567380
...,...,...,...,...,...,...,...,...,...,...,...
3215,72145,Vega Baja Municipio,Puerto Rico,NaN,25580,27791,40.7,53371,52.071350,18.428461,-66.397926
3216,72147,Vieques Municipio,Puerto Rico,NaN,4332,4439,43.6,8771,50.609965,18.122662,-65.439095
3217,72149,Villalba Municipio,Puerto Rico,NaN,11169,11824,38.8,22993,51.424347,18.128155,-66.472816
3218,72151,Yabucoa Municipio,Puerto Rico,NaN,16541,17608,42.5,34149,51.562271,18.070468,-65.896311


In [21]:
cov2 = cov.merge(pop[['population','fips']],how='left', on=["fips"])
cov2['cov_perc'] = (cov2['cases']/cov2['population'])*100
cov2

,fips,county,state,lat,long,date,cases,state_code,deaths,population,cov_perc
0,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-22,0,AL,0,55200.0,0.000000
1,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-23,0,AL,0,55200.0,0.000000
2,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-24,0,AL,0,55200.0,0.000000
3,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-25,0,AL,0,55200.0,0.000000
4,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-26,0,AL,0,55200.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2058667,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-24,881,WY,8,7100.0,12.408451
2058668,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-25,881,WY,8,7100.0,12.408451
2058669,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-26,881,WY,8,7100.0,12.408451
2058670,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-27,891,WY,8,7100.0,12.549296


In [28]:
cvc = pd.read_csv('social_ds.csv')

In [29]:
cvc = cvc[~cvc.eq(-999).any(1)]

In [22]:
cvc.F_TOTAL.value_counts()

0     30973
1     17791
2      7992
3      4570
4      3006
5      2225
6      1842
7      1358
8       977
9       558
10      222
11       59
12        4
13        1
Name: F_TOTAL, dtype: int64

In [32]:
#### changing the name of Country If for fips
cvc.rename(columns={'STCNTY' : 'fips'},inplace=True) 

In [39]:
#### getting the F_TOTAL per COUNTY
cvc_ct = cvc.groupby(['fips']).F_TOTAL.sum().reset_index()
cvc_ct

,fips,F_TOTAL
0,1001,12
1,1003,28
2,1089,56
3,1091,10
4,1093,17
...,...,...
3095,56037,9
3096,56039,1
3097,56041,2
3098,56043,1


In [45]:
cvc_ct.F_TOTAL.quantile(0.7)

13.0

In [46]:
cvc_ct.F_TOTAL.quantile(0.33)

3.0

In [49]:
##### setting the conditions to create the variable VULNERABILITY LEVEL

conditions = [
    (cvc_ct['F_TOTAL'] <= 3),
    (cvc_ct['F_TOTAL'] > 3) & (cvc_ct['F_TOTAL'] <= 13),
    (cvc_ct['F_TOTAL'] >13)]
choices = ['LOW', 'MEDIUM', 'HIGH']
cvc_ct['VULNERA_LEVEL'] = np.select(conditions, choices, default='LOW')

In [50]:
cvc_ct['VULNERA_LEVEL'].value_counts()

MEDIUM    1120
LOW       1075
HIGH       905
Name: VULNERA_LEVEL, dtype: int64

In [51]:
##### Joining Vulnerability level to covid data

cov3 = cov2.merge(cvc_ct[['VULNERA_LEVEL','fips']],how='left', on=["fips"])
cov3

,fips,county,state,lat,long,date,cases,state_code,deaths,population,cov_perc,VULNERA_LEVEL
0,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-22,0,AL,0,55200.0,0.000000,MEDIUM
1,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-23,0,AL,0,55200.0,0.000000,MEDIUM
2,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-24,0,AL,0,55200.0,0.000000,MEDIUM
3,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-25,0,AL,0,55200.0,0.000000,MEDIUM
4,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-26,0,AL,0,55200.0,0.000000,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...
2058667,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-24,881,WY,8,7100.0,12.408451,LOW
2058668,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-25,881,WY,8,7100.0,12.408451,LOW
2058669,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-26,881,WY,8,7100.0,12.408451,LOW
2058670,56045.0,Weston,Wyoming,43.839612,-104.567488,2021-09-27,891,WY,8,7100.0,12.549296,LOW


In [55]:
##### getting covid cases and total population by vulnerability level and date

cov_agg = cov3.groupby(['VULNERA_LEVEL','date']).cases.sum().reset_index()
cov_agg2 = cov3.groupby(['VULNERA_LEVEL','date']).population.sum().reset_index()
cov_agg3 = cov_agg.merge(cov_agg2,how='left', on=["VULNERA_LEVEL",'date'])
cov_agg3['cov_percent'] = (cov_agg3['cases']/cov_agg3['population'])*100
cov_agg3

,VULNERA_LEVEL,date,cases,population,cov_percent
0,HIGH,2020-01-22,1,264339712.0,3.783011e-07
1,HIGH,2020-01-23,1,264339712.0,3.783011e-07
2,HIGH,2020-01-24,2,264339712.0,7.566022e-07
3,HIGH,2020-01-25,2,264339712.0,7.566022e-07
4,HIGH,2020-01-26,5,264339712.0,1.891505e-06
...,...,...,...,...,...
1843,MEDIUM,2021-09-24,5476219,40650089.0,1.347160e+01
1844,MEDIUM,2021-09-25,5488809,40650089.0,1.350258e+01
1845,MEDIUM,2021-09-26,5492587,40650089.0,1.351187e+01
1846,MEDIUM,2021-09-27,5523076,40650089.0,1.358687e+01


In [57]:
import plotly.graph_objs as go
import plotly.express as px
cov_agg3 = cov_agg3[(cov_agg3.date>'2020-04-01') & (cov_agg3.date<'2020-11-01')]
px.line(cov_agg3, x = cov_agg3.date, y = cov_agg3.cov_percent, color='VULNERA_LEVEL',title="Covid Perc")

### COVID ANALYSIS